### This notebook explains how to query the transformer module. "Querying" in this context means to supply a session (defined by items clicked on) to the transformer, and then obtain a session representation (of 100-dimensions). 


You'll may need to pip install several modules:

- pip install keras-pos-embd
- pip install keras_layer_normalization
- pip install keras_transformer
- pip install keras_bert

### IMPORTANT:

Use keras==2.2.3
And tensorflow==1.14.0

In [3]:
! pip install keras-pos-embd
! pip install keras_layer_normalization
! pip install keras_transformer
! pip install keras_bert

You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
! pip install tensorflow==1.14.0

  Using cached https://files.pythonhosted.org/packages/f4/28/96efba1a516cdacc2e2d6d081f699c001d414cc8ca3250e6d59ae657eb2b/tensorflow-1.14.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/91/2d/2ed263449a078cd9c8a9ba50ebd50123adf1f8cfbea1492f9084169b89d9/tensorboard-1.14.0-py3-none-any.whl
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1
You are using pip version 19.0.3, however version 20.2.2 is ava

In [5]:
! pip install keras==2.2.3

  Using cached https://files.pythonhosted.org/packages/06/ea/ad52366ce566f7b54d36834f98868f743ea81a416b3665459a9728287728/Keras-2.2.3-py2.py3-none-any.whl
  Found existing installation: Keras 2.3.0
    Uninstalling Keras-2.3.0:
      Successfully uninstalled Keras-2.3.0
You are using pip version 19.0.3, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import sys
# Make sure the keras_bert_rec folder is in the root of your repo
sys.path.insert(0, './keras_bert_rec')
import importlib
# import keras_bert_rec as rec
# importlib.reload(rec)

import argparse
import keras
import numpy as np
import pandas as pd
import pickle
import sys
import tensorflow as tf
import importlib

from itertools import chain
from keras import backend as K
from keras.models import load_model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [7]:
! pyenv local

3.7.4


### We'll need to specifically point to the "BERT" model (it's the same architecture as BERT, but we didn't train it in order to endow bidirectionality), as well as load in the token dictionary, which maps an item's index to its ID in the Diginetica data. When the data was passed through the transformer module, it was re-indexed. Our output will use the original IDs from Diginetica.

In [ ]:
# Point to the keras_bert_rec module, and the model itself, as well as the token dict and data
KERAS_BERT_REC_PATH = '/home/serena/Personal/AISC/rl_recsys/keras_bert_rec'
DATA_PATH = "/home/serena/Personal/Research/Notebooks/"
MODEL_FILE = \
    DATA_PATH + 'DIGINETICA__30SEQ_LEN_256BATCH_SIZE_2HEAD_NUM_4TRANSFORMER_NUM_50EMBED_DIM_128FEED_FORWARD_DIM_0.1DROPOUT_RATE.model'
TOKEN_DICT_PATH = DATA_PATH + 'diginetica_token_dict.pkl'
TRAIN_SESSION_PATH = DATA_PATH + 'diginetica_train_session_hist.pkl'
TEST_SESSION_PATH = DATA_PATH + 'diginetica_test_session_hist.pkl'

### Some config stuff ###
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.95
# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

### Some functions that we'll need in order to query the model:

In [ ]:
def get_lookup(item_id):
    """ If an item id isn't in the dictionary,
        give it the id 1."""
    
    try:
        mapped_id = lookup_dict[item_id]
    except KeyError:
        mapped_id = lookup_dict[1]
        
    return mapped_id

def apply_lookup(row, row_name):
        
    if row_name == "prev_items":

        item_ids_list = [get_lookup(x) for x in row["prev_items"]]

    elif row_name == "prev_sessions":

        item_ids_list = []

        sess = row["prev_sessions"]
        sess_list = []

        for i in sess:
            item_ids_list = [lookup_dict[x] for x in i]
            item_ids_list.append(item_ids_list)
    
    elif row_name == "target":
        
        item_ids_list = [get_lookup(row["target"])]

    
    return item_ids_list


def get_session_rep_model(
    mlm_model, token_dict, head_num, transformer_num, 
    embed_dim, feed_forward_dim,  seq_len):
    # create duplicate model that returns session representation, rather than cloze task/masked scentence
    # note that the model architecture needs to be the same!
    
    q = rec.get_rec_model(
        token_num=len(token_dict),
        head_num=head_num,
        transformer_num=transformer_num,
        embed_dim=embed_dim,
        feed_forward_dim=feed_forward_dim,
        seq_len=seq_len,
        pos_num=seq_len,
        dropout_rate=0.05,
        training=False
    )
    model = keras.models.Model(inputs=q[0], outputs=q[1])
    
    # copy layer weights up to the session representation
    for i in range(len(model.layers)):
        model.layers[i].set_weights(mlm_model.layers[i].get_weights())
    
    return model

def clean_session(sequence, seq_len, token_map):
    """Map tokens from raw values to embedding indexes and pad.
    
    sequence: list or array of item ids
    token_map: dict where keys are raw item ids, values are corresponding embedding index
    seq_len: length of sequence defined for model
    """
    # Copy, or else it'll overwrite the dataframe
    clean_sequence = sequence.copy()
    for j, token in enumerate(clean_sequence):
            clean_sequence[j] = token_map.get(token, 0)
    if len(clean_sequence) < seq_len:
            clean_sequence = clean_sequence + [0] * (seq_len - len(clean_sequence))
    # If a sequence is greater than SEQ_LEN then take the most recent SEQ_LEN clicks
    return clean_sequence[-seq_len:]


### We'll have to specify the parameters of the transformer as well.

In [ ]:
# These parameters match the model specified by MODEL_FILE
SEQ_LEN = 30
BATCH_SIZE = 256
HEAD_NUM = 2
TRANSFORMER_NUM = 4
EMBEDDING_DIMENSION = 50
FEED_FORWARD_DIM = 128
DROPOUT_RATE= 0.1

### And now we'll load in the actual model, with the required parameters.

In [ ]:
bert_model = load_model(MODEL_FILE, custom_objects=rec.get_custom_objects())
model = get_session_rep_model(
    bert_model, lookup_dict, HEAD_NUM, TRANSFORMER_NUM, EMBEDDING_DIMENSION, FEED_FORWARD_DIM, SEQ_LEN)

### Let's also load in that lookup dictionary. We'll also need to get the embedding layer.

In [ ]:
lookup_dict = pd.read_pickle(TOKEN_DICT_PATH)

In [ ]:
embeddings = model.layers[1].get_weights()[0]
embeddings

### The functions below will be used to query the transformer.

In [ ]:
def get_session_representation(row, token_dict, model, seq_len=30):
    
    clean_representation = clean_session(row["session"], seq_len, token_dict)
    # Get flattened representation
    session_representation = model.predict(np.array([clean_representation])).flatten()
    
    if len(clean_representation) == 0:
        return np.array([0] * EMBEDDING_DIMENSION)
    
    else:
    
        return session_representation

def create_df(df, token_dict, model):
        
    df["session_representation"] = \
        df.apply(lambda x: get_session_representation(x,token_dict, model), axis=1)
      
    return df

### Now we can obtain our session representations!

In [ ]:
train_df = pd.read_pickle(TRAIN_SESSION_PATH)
test_df = pd.read_pickle(TEST_SESSION_PATH)

# Just to show that it works!
train_data = create_df(train_df[:10], lookup_dict, model)
#test_data = create_df(test_df, lookup_dict, model)